# DSCI552: HW7
#Part 1: Generative Models for Text 


***Name: Cheng Peng***


***USC ID: 6898-9638-37***

In [ ]:
import os 
import tensorflow as tf 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import LSTM 
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint 
from tensorflow.keras.utils import to_categorical
import string
import numpy as np


# A):  


In this problem, we are trying to build a generative model to mimic the writing style of prominent British Mathematician, Philosopher, prolific writer, and political activist, Bertrand Russell.

# B): Download the data and prepare the corpus 

***Comment:***I am using google collab with 12.72 GB of RAM. I tired to include all books in the corpus but it used up all the memory. I decided to only use the first 4 books as my corpus. 

In [3]:
#base_directory = "/content/drive/MyDrive/DSCI552/Homework 7 Data/Book Files/books"
base_directory = "../data/Book Files/books"
files = os.listdir(base_directory)
corpus = open("corpus.txt", "w")
for file in files:
    print("write {} to the corpus".format(file))
    with open(base_directory+"/"+file, 'r', encoding='ascii', errors='ignore') as datastream:
        for line in datastream:
            corpus.write(line)
    
corpus.close()

write MLOE.txt to the corpus
write OKEWFSMP.txt to the corpus
write TPP.txt to the corpus
write TAM.txt to the corpus


# C): LSTM

I): Concatenate your text files to create a corpus of Russell’s writings.

In [ ]:
# load the book 
corpus = open("corpus.txt", "r", encoding = "ascii").read()
print("There are {} characters in the corpus.".format(len(corpus)))

There are 1576925 characters in the corpus.


II): Convert to character level representation

In [ ]:
# find the unique characters in the corpus 
# convert to all lower cases and remove the punctuations 
punctuations = string.punctuation
corpus = corpus.lower()
corpus = corpus.translate(str.maketrans('', '', punctuations))
chars = sorted(list(set(corpus)))

In [ ]:
print("There are {} unique characters in the corpus.".format(len(chars)))
print(chars)

There are 38 unique characters in the corpus.
['\n', ' ', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [ ]:
# creat mapping from character to integer and integer to character 
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))

III): Choose a window size W

IV): Construct input and output using the sliding window 

In [ ]:
# Set the sliding window size = W
# i want the length of each input sequence to be 100, thus set W to 101 
W = 101 
inputs, outputs = [], []
for i in range(0, len(corpus)-W-1):
    x = corpus[i:i+W-1]
    y = corpus[i+W-1]
    # convert the characters to integer 
    inputs.append([char_to_int[c] for c in x])
    outputs.append(char_to_int[y])


In [ ]:
len(inputs)

1532602

In [ ]:
len(outputs)

1532602

In [ ]:
print("There are {} input sequences and each sequence is of length {}".format(len(inputs), len(inputs[0])))

There are 1532602 input sequences and each sequence is of length 100


V): One-hot encoding on the output 

In [ ]:
# use the to_categorical function to convert y to one hot endcoding 
outputs = to_categorical(outputs, num_classes=len(chars))

In [ ]:
print("The output has been converted to a matrix of shape {}, each output character is encoded using one-hot encoding. ".format(outputs.shape))

The output has been converted to a matrix of shape (1532602, 38), each output character is encoded using one-hot encoding. 


In [ ]:
print("For instance, output character is {} is encoded as \n{}".format(int_to_char[np.where(outputs[5]==1)[0][0]], outputs[5]))

For instance, output character is h is encoded as 
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [ ]:
inputs = np.array(inputs)


In [ ]:
# reshape the input dimension to be [num_samples, sequence_length, features]
inputs = np.expand_dims(inputs, axis=2)

VI): Single hidden layer for the LSTM with N=256 memory units 

VII): Softmax output layer and cross entropy as the loss function 

In [ ]:
# normalize inputs to be between [0, 1]
inputs = inputs/float(len(chars)-1)
X = inputs
y = outputs

In [ ]:
X.shape[1], X.shape[2]

(100, 1)

In [ ]:
# define a model 
model = Sequential()
model.add(LSTM(256, input_shape = (X.shape[1], X.shape[2])))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 256)               264192    
_________________________________________________________________
dense_3 (Dense)              (None, 38)                9766      
Total params: 273,958
Trainable params: 273,958
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.input_shape

(None, 100, 1)

In [ ]:
model.output_shape

(None, 100, 38)

VIII): Use the entire dataset to train the model 

IX): Training 

X): Use model check-point to keep the network weight, find the best set of weight in terms of loss

In [ ]:
# define the chackpoints 
model_file = "model_{epoch:02d}_{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(model_file, monitor='loss', verbose=1, save_best_only=True)
callback = [checkpoint]

In [ ]:
model.fit(X, y, epochs=30, batch_size = 128, callbacks=callback, verbose=1)

Epoch 1/30
11972/11974 [============================>.] - ETA: 0s - loss: 2.4856
Epoch 00001: loss improved from inf to 2.48559, saving model to model_01_2.4856.hdf5
11974/11974 [==============================] - 154s 13ms/step - loss: 2.4856
Epoch 2/30
11973/11974 [============================>.] - ETA: 0s - loss: 2.0814
Epoch 00002: loss improved from 2.48559 to 2.08144, saving model to model_02_2.0814.hdf5
11974/11974 [==============================] - 155s 13ms/step - loss: 2.0814
Epoch 3/30
11974/11974 [==============================] - ETA: 0s - loss: 1.9020
Epoch 00003: loss improved from 2.08144 to 1.90201, saving model to model_03_1.9020.hdf5
11974/11974 [==============================] - 155s 13ms/step - loss: 1.9020
Epoch 4/30
11973/11974 [============================>.] - ETA: 0s - loss: 1.7973
Epoch 00004: loss improved from 1.90201 to 1.79726, saving model to model_04_1.7973.hdf5
11974/11974 [==============================] - 155s 13ms/step - loss: 1.7973
Epoch 5/30
11974

XI): Use the netowrk with the best weights to generate 1000 characters. 

Input: There are those who take mental phenomena naively, just as they
would physical phenomena. This school of psychologists tends not to
emphasize the object.

In [ ]:
# use the best model to generate text 
best_model = "/content/model_30_1.3428.hdf5"
model.load_weights(best_model)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
# encode the sample given 
input_text = "There are those who take mental phenomena naively, just as they would physical phenomena. This school of psychologists tends not to emphasize the object.".lower()
input_text = input_text.translate(str.maketrans('', '', punctuations))
input_sequence = [char_to_int[c] for c in input_text]


In [ ]:
# generate 1000 characters 
for i in range(1000):
    X = np.reshape(input_sequence, (1, len(input_sequence), 1))
    X = X/float(len(chars)-1)
    predict = model.predict(X, verbose=0)
    index = np.argmax(predict)
    char_predict = int_to_char[index]
    input_text+=char_predict
    input_sequence.append(index)
    input_sequence = input_sequence[1:]



In [ ]:
input_text

'there are those who take mental phenomena naively just as they would physical phenomena this school of psychologists tends not to emphasize the objective of the perspective shat in the sense of the sensedata which is the semsation of the sensedata which is the semsation of the sensedata which is the semsation of the sensedata which is the semsation of the sensedata which is the semsation of the sensedata which is the semsation of the sensedata which is the semsation of the sensedata which is the semsation of the sensedata which is the semsation of the sensedata which is the semsation of the sensedata which is the semsation of the sensedata which is the semsation of the sensedata which is the semsation of the sensedata which is the semsation of the sensedata which is the semsation of the sensedata which is the semsation of the sensedata which is the semsation of the sensedata which is the semsation of the sensedata which is the semsation of the sensedata which is the semsation of the s

***Comment:***


* The model is able to generate sentences that confroms to the line format of the original text 
* The generated characters are grouped into "words", some words are correctlt spelled which means the model learns some spelling.   
* The model keep generating the same sentence ("which is the semsation of the sensedata"), the reason can be lack of varity in the training samples, and small sample size.  
*   Since all the punctuation are removed in the traning set, the mode cannot generate any punctuation marks. 




XII): More training with a deeper model 
 

# References:
1. https://machinelearningmastery.com/text-generation-lstm-recurrent-neural-networks-python-keras/